In [0]:
import pandas as pd
import sklearn.linear_model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
#1. Загрузка файлов и перименование полей
ratings = pd.read_csv('u.data.csv', sep="\t", header=None)
ratings = ratings.rename(
               columns={
                   0:"user id",
                   1:"item id",
                   2:"rating",
                   3:"timestamp"
               }
)
movies = pd.read_csv('u.item.csv', sep="|", header=None)
movies = movies.rename(
               columns={
                   0:"movie id",
                   1:"movie title",
                   2:"release date",
                   3:"video release date",
                   4:"IMDb URL",
                   5:"unknown",
                   6:"Action",
                   7:"Adventure",
                   8:"Animation",
                   9:"Children's",
                   10:"Comedy",
                   11:"Crime",
                   12:"Documentary",
                   13:"Drama",
                   14:"Fantasy",
                   15:"Film-Noir",
                   16:"Horror",
                   17:"Musica",
                   18:"Mystery",
                   19:"Romance",
                   20:"Sci-Fi",
                   21:"Thriller",
                   22:"War",
                   23:"Western"
               }
)

In [4]:
#2. ID пользователя с наибольшим кол-ом оценок
top_user = ratings.groupby('user id').count().sort_values('item id', ascending=False).reset_index()[:1]['user id']
top_user

0    405
Name: user id, dtype: int64

In [5]:
#3. Фильмы, которые выбра top_user
subset_ratings = ratings[ratings['user id'] == top_user.values[0]]
subset_ratings.head()

,user id,item id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


In [0]:
#4. Добавление полей
subset_ratings_join = subset_ratings.merge(movies, 
                     how='left', left_on = 'item id', right_on = 'movie id')

In [0]:
subset_ratings_join['Year'] = subset_ratings_join['release date'].str.split("-", expand = True)[2]

In [0]:
all_ratings = ratings.groupby('item id').agg(['sum','count'])['rating'].reset_index()
all_ratings = all_ratings.rename(
               columns={
                   "sum":"Sum ratings",
                   "count":"Count ratings"})

In [0]:
subset_ratings_join = subset_ratings_join.merge(all_ratings, how = 'left', on='item id')

In [0]:
subset_ratings_join = subset_ratings_join.drop(['movie id', 'video release date', 'IMDb URL', 'timestamp',
                         'movie title', 'release date', 'user id', 'item id'], axis=1)

In [11]:
subset_ratings_join.head()

,rating,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musica,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year,Sum ratings,Count ratings
0,4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1994,1600,394
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1995,30,9
2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1947,1,1
3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1991,252,65
4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1995,108,32


In [0]:
rating = subset_ratings_join['rating']
variables = subset_ratings_join.drop(['rating'], axis=1)
model = LinearRegression()

In [0]:
#5. Трейн-тест-сплитинг
X_train, X_test, y_train, y_test = train_test_split(variables, rating, test_size=0.2)

In [0]:
sc = sklearn.preprocessing.StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [50]:
#6. Линейная регрессия
result = model.fit(X_train, y_train)

print('intercept:', model.intercept_)
print('coef_variables:', model.coef_)

intercept: 1.845500848896434
coef_variables: [ 0.         -0.06334763 -0.08478513 -0.04871727  0.11529698  0.08517349
  0.0078396   0.04381998  0.08648942 -0.00656908 -0.09110166  0.00459067
  0.07949081  0.02153199 -0.08568001 -0.06308892  0.06309737 -0.05752936
  0.01617355 -0.08549796 -0.53795259  1.29525224]


In [51]:
#7. Качество модели
y_hat = model.predict(X_test).reshape(-1,1)

sklearn.metrics.r2_score(y_test, y_hat)

0.3987615428066025

In [0]:
# Вывод: R2, при многоразовых перезапусках, в диапазоне от 0.2 до 0.5. Можно сказать, что это средний показатель качества по прогнозу оценки пользователя.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
#8. Загрузка данных в Spark и переименование колонко
s_ratings = spark.read.csv('u.data.csv', header=False, inferSchema=True, sep="\t")
s_ratings = s_ratings.withColumnRenamed('_c0', 'user_id')
s_ratings = s_ratings.withColumnRenamed('_c1', 'movie_id')
s_ratings = s_ratings.withColumnRenamed('_c2', 'rating')
s_ratings = s_ratings.withColumnRenamed('_c3', 'timestamp')

In [0]:
s_movies = spark.read.csv('u.item.csv', header=False, inferSchema=True, sep="|")
s_movies=s_movies.withColumnRenamed('_c0','movie_id')
s_movies=s_movies.withColumnRenamed('_c1','movie_title')
s_movies=s_movies.withColumnRenamed('_c2','release_date')
s_movies=s_movies.withColumnRenamed('_c3','video_release_date')
s_movies=s_movies.withColumnRenamed('_c4','IMDb_URL')
s_movies=s_movies.withColumnRenamed('_c5','unknown')
s_movies=s_movies.withColumnRenamed('_c6','Action')
s_movies=s_movies.withColumnRenamed('_c7','Adventure')
s_movies=s_movies.withColumnRenamed('_c8','Animation')
s_movies=s_movies.withColumnRenamed('_c9','Childrens')
s_movies=s_movies.withColumnRenamed('_c10','Comedy')
s_movies=s_movies.withColumnRenamed('_c11','Crime')
s_movies=s_movies.withColumnRenamed('_c12','Documentary')
s_movies=s_movies.withColumnRenamed('_c13','Drama')
s_movies=s_movies.withColumnRenamed('_c14','Fantasy')
s_movies=s_movies.withColumnRenamed('_c15','Film-Noir')
s_movies=s_movies.withColumnRenamed('_c16','Horror')
s_movies=s_movies.withColumnRenamed('_c17','Musica')
s_movies=s_movies.withColumnRenamed('_c18','Mystery')
s_movies=s_movies.withColumnRenamed('_c19','Romance')
s_movies=s_movies.withColumnRenamed('_c20','Sci-Fi')
s_movies=s_movies.withColumnRenamed('_c21','Thriller')
s_movies=s_movies.withColumnRenamed('_c22','War')
s_movies=s_movies.withColumnRenamed('_c23','Western')

In [157]:
s_ratings.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [158]:
s_movies.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musica|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|     0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|        

In [0]:
#9 Средняя оценка для каждого фильма

In [0]:
avg_s_ratings = s_ratings.groupBy('movie_id').avg('rating')

In [0]:
s_movies = s_movies.join(avg_s_ratings, 'movie_id', how='left')

In [162]:
s_movies.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+------------------+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musica|Mystery|Romance|Sci-Fi|Thriller|War|Western|       avg(rating)|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+------------------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|     0|      0|      0|     0|       0|  0|    

In [0]:
#10. Средняя оценка для каждого жанра

In [0]:
sum_s_ratings = s_ratings.groupBy('movie_id').sum('rating')
cnt_s_ratings = s_ratings.groupBy('movie_id').count()

In [0]:
s_movies = s_movies.join(sum_s_ratings, 'movie_id', how='left')
s_movies = s_movies.join(cnt_s_ratings, 'movie_id', how='left')

In [182]:
melt_s_movies = s_movies.drop('movie_id')
melt_s_movies = melt_s_movies.drop('movie_title')
melt_s_movies = melt_s_movies.drop('release_date')
melt_s_movies = melt_s_movies.drop('video_release_date')
melt_s_movies = melt_s_movies.drop('IMDb_URL')
melt_s_movies = melt_s_movies.drop('avg(rating)')
melt_s_movies.show()

+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+-----------+-----+
|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musica|Mystery|Romance|Sci-Fi|Thriller|War|Western|sum(rating)|count|
+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+-----------+-----+
|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|     0|      0|      0|     0|       0|  0|      0|       1753|  452|
|      0|     1|        1|        0|        0|     0|    0|          0|    0|      0|        0|     0|     0|      0|      0|     0|       1|  0|      0|        420|  131|
|      0|     0|        0|        0|        0|     0|    0|          0|    0|      0|        0|     0|     0|      0|      0|     0|       1

In [0]:
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable 

In [0]:
def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

In [0]:
melt_s_movies = melt(melt_s_movies, id_vars=['sum(rating)', 'count'], value_vars=['unknown', 'Action', 'Adventure',
                                                         'Animation', 'Childrens', 'Comedy',
                                                         'Crime', 'Documentary', 'Drama',
                                                         'Fantasy', 'Film-Noir', 'Horror',
                                                         'Musica', 'Mystery', 'Romance',
                                                         'Sci-Fi', 'Thriller', 'War',
                                                         'Western'])

In [0]:
melt_s_movies = melt_s_movies.filter(melt_s_movies.value > 0)

In [189]:
melt_s_movies.show()

+-----------+-----+---------+-----+
|sum(rating)|count| variable|value|
+-----------+-----+---------+-----+
|       1753|  452|Animation|    1|
|       1753|  452|Childrens|    1|
|       1753|  452|   Comedy|    1|
|        420|  131|   Action|    1|
|        420|  131|Adventure|    1|
|        420|  131| Thriller|    1|
|        273|   90| Thriller|    1|
|        742|  209|   Action|    1|
|        742|  209|   Comedy|    1|
|        742|  209|    Drama|    1|
|        284|   86|    Crime|    1|
|        284|   86|    Drama|    1|
|        284|   86| Thriller|    1|
|         93|   26|    Drama|    1|
|       1489|  392|    Drama|    1|
|       1489|  392|   Sci-Fi|    1|
|        875|  219|Childrens|    1|
|        875|  219|   Comedy|    1|
|        875|  219|    Drama|    1|
|       1165|  299|    Drama|    1|
+-----------+-----+---------+-----+
only showing top 20 rows



In [0]:
melt1 = melt_s_movies.groupBy('variable').sum('sum(rating)')

In [0]:
melt2 = melt_s_movies.groupBy('variable').sum('count')

In [0]:
ratings_by_genres = melt1.join(melt2, 'variable', how='left')

In [0]:
ratings_by_genres = ratings_by_genres.withColumn("Average rating", col('sum(sum(rating))') / col('sum(count)'))

In [203]:
ratings_by_genres.show()

+-----------+----------------+----------+------------------+
|   variable|sum(sum(rating))|sum(count)|    Average rating|
+-----------+----------------+----------+------------------+
|      Crime|           29258|      8055|3.6322780881440098|
|    Romance|           70482|     19461| 3.621704948358255|
|   Thriller|           76749|     21872|3.5090069495245064|
|  Adventure|           48184|     13753| 3.503526503308369|
|    unknown|              32|        10|               3.2|
|      Drama|          147108|     39895|3.6873793708484772|
|        War|           35861|      9398| 3.815811874866993|
|Documentary|            2784|       758|3.6728232189973613|
|    Fantasy|            4347|      1352|3.2152366863905324|
|     Musica|           17445|      4954| 3.521396851029471|
|    Mystery|           19082|      5245|  3.63813155386082|
|  Animation|           12894|      3605|3.5766990291262135|
|  Film-Noir|            6796|      1733|3.9215233698788228|
|     Horror|           

In [0]:
#11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами
#(по количеству оценок, либо по самой оценке - на Ваш выбор)

In [0]:
top_5 = s_movies
bottom_5 = s_movies

In [225]:
top_5.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+------------------+-----------+-----+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musica|Mystery|Romance|Sci-Fi|Thriller|War|Western|       avg(rating)|sum(rating)|count|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+------------------+-----------+-----+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|

In [241]:
top_5 = top_5.sort('avg(rating)', ascending=False)
top_5 = top_5.limit(5)
top_5.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+-----------+-----------+-----+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musica|Mystery|Romance|Sci-Fi|Thriller|War|Western|avg(rating)|sum(rating)|count|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+-----------+-----------+-----+
|     814|Great Day in Harl...| 01-Jan-1994|              null|http://us.imdb.co...|      0|     0|        0|        0|        0|     0|    0|          1|    0|      0|        0|     0|     0|      0

In [242]:
bottom_5 = bottom_5.sort('avg(rating)', ascending=True)
bottom_5 = bottom_5.limit(5)
bottom_5.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+-----------+-----------+-----+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musica|Mystery|Romance|Sci-Fi|Thriller|War|Western|avg(rating)|sum(rating)|count|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+------+-------+-------+------+--------+---+-------+-----------+-----------+-----+
|     599|Police Story 4: P...| 16-Aug-1996|              null|http://us.imdb.co...|      0|     1|        0|        0|        0|     0|    0|          0|    0|      0|        0|     0|     0|      0